In [12]:
import pandas as pd
from glob import glob
import json
from copy import copy

# Load Metadata

In [13]:
sample_info_df = pd.read_excel('../FullRBPsWithEnzymes/Hugo_Samples_IDs.xlsx')
sample_info_df = sample_info_df.dropna()

id_to_label = {}

for r in sample_info_df.iterrows():
    r = r[1]
    
    label = r.Construct
    sample_id = r.Sample.replace('_', '-')
    
    if 'SLBP' not in label and 'RPS2' not in label:
        id_to_label[sample_id] = label
id_to_label

{'Hugo-A1': 'FOX2-APOBEC1_R1',
 'Hugo-A2': 'FOX2-APOBEC1_R2',
 'Hugo-A3': 'FOX2-APOBEC1_R3',
 'Hugo-B2': 'FOX2-p0078_R1',
 'Hugo-B3': 'FOX2-P0078_R2',
 'Hugo-B4': 'FOX2-P0078_R3',
 'Hugo-B5': 'FOX2-8e_R1',
 'Hugo-B6': 'FOX2-8e_R2',
 'Hugo-B7': 'FOX2-8e_R3',
 'Hugo-D4': 'APOBEC1_alone_R1',
 'Hugo-D5': 'APOBEC1_alone_R2',
 'Hugo-D6': 'APOBEC1_alone_R3',
 'Hugo-D7': '8e_alone_R1',
 'Hugo-D8': '8e_alone_R2',
 'Hugo-E1': '8e_alone_R3',
 'Hugo-E2': 'p0078_alone_R1',
 'Hugo-E3': 'p0078_alone_R2',
 'Hugo-E4': 'p0078_alone_R3',
 'Hugo-E5': 'MinA_alone_R1',
 'Hugo-E6': 'MinA_alone_R2',
 'Hugo-E7': 'MinA_alone_R3',
 'Hugo-E8': 'FOX2-MinA_R1',
 'Hugo-F1': 'FOX2-MinA_R2',
 'Hugo-F2': 'FOX2-MinA_R3'}

In [14]:
len(id_to_label)

24

# Load JSON template to fill in 

In [15]:
glob('/projects/ps-yeolab4/ekofman/Hugo/RBFOX2_sailor_outs/*.bed')

['/projects/ps-yeolab4/ekofman/Hugo/RBFOX2_sailor_outs/Hugo-E6Aligned.sortedByCoord.out.bam.combined.readfiltered.formatted.varfiltered.snpfiltered.ranked.bed',
 '/projects/ps-yeolab4/ekofman/Hugo/RBFOX2_sailor_outs/Hugo-A2Aligned.sortedByCoord.out.bam.combined.readfiltered.formatted.varfiltered.snpfiltered.ranked.bed',
 '/projects/ps-yeolab4/ekofman/Hugo/RBFOX2_sailor_outs/Hugo-B6Aligned.sortedByCoord.out.bam.combined.readfiltered.formatted.varfiltered.snpfiltered.ranked.bed',
 '/projects/ps-yeolab4/ekofman/Hugo/RBFOX2_sailor_outs/Hugo-F2Aligned.sortedByCoord.out.bam.combined.readfiltered.formatted.varfiltered.snpfiltered.ranked.bed',
 '/projects/ps-yeolab4/ekofman/Hugo/RBFOX2_sailor_outs/Hugo-E4Aligned.sortedByCoord.out.bam.combined.readfiltered.formatted.varfiltered.snpfiltered.ranked.bed',
 '/projects/ps-yeolab4/ekofman/Hugo/RBFOX2_sailor_outs/Hugo-B4Aligned.sortedByCoord.out.bam.combined.readfiltered.formatted.varfiltered.snpfiltered.ranked.bed',
 '/projects/ps-yeolab4/ekofman/Hug

In [16]:
from glob import glob
from collections import defaultdict

label_to_sailor_outfile_ct = defaultdict(lambda:{})

# sailor edits
sailor_outs_ct = glob('/projects/ps-yeolab4/ekofman/Hugo/RBFOX2_sailor_outs/*.bed')
# bams and bw
fwd_bw_ct = glob('/projects/ps-yeolab4/ekofman/Hugo/RBFOX2_sailor_outs/8_bw_and_bam/*fwd*.bw')
rev_bw_ct = glob('/projects/ps-yeolab4/ekofman/Hugo/RBFOX2_sailor_outs/8_bw_and_bam/*rev*.bw')
final_bam_ct =  glob('/projects/ps-yeolab4/ekofman/Hugo/RBFOX2_sailor_outs/8_bw_and_bam/*sorted.bam')

labels_present = []
for f in sailor_outs_ct:
    sample_id = f.split('/')[-1].split('Aligned')[0]
    try:
        label = id_to_label[sample_id]
        labels_present.append(label)
        label_to_sailor_outfile_ct[label]['sites_file'] = f
    except Exception as e:
        print(e)
    
print(set(labels_present).difference(set(id_to_label.values())))
print(set(id_to_label.values()).difference(labels_present))

for f in fwd_bw_ct:
    sample_id = f.split('/')[-1].split('Aligned')[0]
    try:
        label = id_to_label[sample_id]
        labels_present.append(label)
        label_to_sailor_outfile_ct[label]['fwd_bw'] = f
    except Exception as e:
        print(e)
    
for f in rev_bw_ct:
    sample_id = f.split('/')[-1].split('Aligned')[0]
    try:
        label = id_to_label[sample_id]
        labels_present.append(label)
        label_to_sailor_outfile_ct[label]['rev_bw'] = f
    except Exception as e:
        print(e)
        
for f in final_bam_ct:
    sample_id = f.split('/')[-1].split('Aligned')[0]
    try:
        label = id_to_label[sample_id]
        labels_present.append(label)
        label_to_sailor_outfile_ct[label]['final_bam'] = f
    except Exception as e:
        print(e)

set()
set()
'Hugo-A7'
'Hugo-C6'
'Hugo-C4'
'Hugo-B1'
'Hugo-C4'
'Hugo-B1'
'Hugo-C6'
'Hugo-A7'
'Hugo-C6'
'Hugo-B1'
'Hugo-C4'
'Hugo-A7'


In [17]:
with open('4_SLBP_peakcalling_inputs/peakcalling_inputs_template.json', 'r') as f:
    json_template_ct = json.load(f)
    
json_template_ct['output_folder'] = '/projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/4.12_RBFOX2_peakcalling_outputs_ct/'
json_template_ct['fdr_threshold'] = 0.1
json_template_ct['max_merge_dist'] = 15
json_template_ct['edit_type'] = 'ct'
json_template_ct

label_to_json_ct = {}
label_to_json_path_ct = {}
for label in id_to_label.values():
    print(label)

    json_for_sample_ct = copy(json_template_ct)
    json_for_sample_ct['label'] = label
    json_for_sample_ct['stamp_sites_file'] = label_to_sailor_outfile_ct.get(label).get('sites_file')
    json_for_sample_ct['forward_bw'] = label_to_sailor_outfile_ct.get(label).get('fwd_bw')
    json_for_sample_ct['reverse_bw'] = label_to_sailor_outfile_ct.get(label).get('rev_bw')
    json_for_sample_ct['bam'] = label_to_sailor_outfile_ct.get(label).get('final_bam')
    label_to_json_ct[label] = json_for_sample_ct
    
    json_path = '4_RBFOX2_peakcalling_inputs/CT_{}_peakcalling.json'.format(label)
    label_to_json_path_ct[label] = json_path
    
    with open(json_path, 'w') as f:
        json.dump(json_for_sample_ct, f)

FOX2-APOBEC1_R1
FOX2-APOBEC1_R2
FOX2-APOBEC1_R3
FOX2-p0078_R1
FOX2-P0078_R2
FOX2-P0078_R3
FOX2-8e_R1
FOX2-8e_R2
FOX2-8e_R3
APOBEC1_alone_R1
APOBEC1_alone_R2
APOBEC1_alone_R3
8e_alone_R1
8e_alone_R2
8e_alone_R3
p0078_alone_R1
p0078_alone_R2
p0078_alone_R3
MinA_alone_R1
MinA_alone_R2
MinA_alone_R3
FOX2-MinA_R1
FOX2-MinA_R2
FOX2-MinA_R3


# AI

In [18]:
from glob import glob
from collections import defaultdict

label_to_sailor_outfile_ai = defaultdict(lambda:{})

# sailor edits
sailor_outs_ai = glob('/projects/ps-yeolab4/ekofman/Hugo/RBFOX2_sailor_outs_ai/*.bed')
# bams and bw
fwd_bw_ai = glob('/projects/ps-yeolab4/ekofman/Hugo/RBFOX2_sailor_outs_ai/8_bw_and_bam/*fwd*.bw')
rev_bw_ai = glob('/projects/ps-yeolab4/ekofman/Hugo/RBFOX2_sailor_outs_ai/8_bw_and_bam/*rev*.bw')
final_bam_ai =  glob('/projects/ps-yeolab4/ekofman/Hugo/RBFOX2_sailor_outs_ai/8_bw_and_bam/*sorted.bam')

labels_present = []
for f in sailor_outs_ai:
    sample_id = f.split('/')[-1].split('Aligned')[0]
    try:
        label = id_to_label[sample_id]
        labels_present.append(label)
        label_to_sailor_outfile_ai[label]['sites_file'] = f
    except Exception as e:
        print(e)
    
print(set(labels_present).difference(set(id_to_label.values())))
print(set(id_to_label.values()).difference(labels_present))

for f in fwd_bw_ai:
    sample_id = f.split('/')[-1].split('Aligned')[0]
    try:
        label = id_to_label[sample_id]
        labels_present.append(label)
        label_to_sailor_outfile_ai[label]['fwd_bw'] = f
    except Exception as e:
        print(e)
    
for f in rev_bw_ai:
    sample_id = f.split('/')[-1].split('Aligned')[0]
    try:
        label = id_to_label[sample_id]
        labels_present.append(label)
        label_to_sailor_outfile_ai[label]['rev_bw'] = f
    except Exception as e:
        print(e)
        
for f in final_bam_ai:
    sample_id = f.split('/')[-1].split('Aligned')[0]
    try:
        label = id_to_label[sample_id]
        labels_present.append(label)
        label_to_sailor_outfile_ai[label]['final_bam'] = f
    except Exception as e:
        print(e)

set()
set()
'Hugo-F7'
'Hugo-A6'
'Hugo-A5'
'Hugo-C3'
'Hugo-A8'
'Hugo-D3'
'Hugo-F8'
'Hugo-B1'
'Hugo-A8'
'Hugo-C3'
'Hugo-F8'
'Hugo-B1'
'Hugo-D3'
'Hugo-F7'
'Hugo-A5'
'Hugo-A6'
'Hugo-A8'
'Hugo-C3'
'Hugo-A6'
'Hugo-A5'
'Hugo-F8'
'Hugo-F7'
'Hugo-B1'
'Hugo-D3'


In [19]:
len(label_to_sailor_outfile_ai)

24

In [20]:
with open('4_SLBP_peakcalling_inputs/peakcalling_inputs_template.json', 'r') as f:
    json_template_ai = json.load(f)
    
json_template_ai['output_folder'] = '/projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/4.22_RBFOX2_peakcalling_outputs_ai/'
json_template_ai['fdr_threshold'] = 0.1
json_template_ai['max_merge_dist'] = 15
json_template_ai['edit_type'] = 'ai'
json_template_ai

label_to_json_ai = {}
label_to_json_path_ai = {}
for label in id_to_label.values():
    print(label)

    json_for_sample_ai = copy(json_template_ai)
    json_for_sample_ai['label'] = label
    json_for_sample_ai['stamp_sites_file'] = label_to_sailor_outfile_ai.get(label).get('sites_file')
    json_for_sample_ai['forward_bw'] = label_to_sailor_outfile_ai.get(label).get('fwd_bw')
    json_for_sample_ai['reverse_bw'] = label_to_sailor_outfile_ai.get(label).get('rev_bw')
    json_for_sample_ai['bam'] = label_to_sailor_outfile_ai.get(label).get('final_bam')
    label_to_json_ai[label] = json_for_sample_ai
    
    json_path = '4_RBFOX2_peakcalling_inputs/AI_{}_peakcalling.json'.format(label)
    label_to_json_path_ai[label] = json_path
    
    with open(json_path, 'w') as f:
        json.dump(json_for_sample_ai, f)

FOX2-APOBEC1_R1
FOX2-APOBEC1_R2
FOX2-APOBEC1_R3
FOX2-p0078_R1
FOX2-P0078_R2
FOX2-P0078_R3
FOX2-8e_R1
FOX2-8e_R2
FOX2-8e_R3
APOBEC1_alone_R1
APOBEC1_alone_R2
APOBEC1_alone_R3
8e_alone_R1
8e_alone_R2
8e_alone_R3
p0078_alone_R1
p0078_alone_R2
p0078_alone_R3
MinA_alone_R1
MinA_alone_R2
MinA_alone_R3
FOX2-MinA_R1
FOX2-MinA_R2
FOX2-MinA_R3


# qsub commands

In [41]:
import os

header_lines = [
    '#!/bin/bash', 
    '#PBS -N peaks_ct_rbfox',
    '#PBS -o peaks_ct_rbfox.sh.out',
    '#PBS -e peaks_ct_rbfox.sh.err',
    '#PBS -V',
    '#PBS -l walltime=10:00:00',
    '#PBS -l nodes=1:ppn=1',
    '#PBS -A yeo-group',
    '#PBS -q home',
    '#PBS -t 1-{}'.format(len(label_to_json_path_ct)),
    '',
    '# Go to the directory from which the script was called',
    'cd /projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/4.3_peakcalling_logs',
    ''
]

command = 'cmd[{}]="module load stamp; snakemake --rerun-incomplete \
--profile /projects/ps-yeolab3/ekofman/sc_STAMP_pipeline/STAMP/profiles/tscc_peakcaller_temp/ \
--configfile /projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/{};"'

commands = []

for i, f in enumerate(label_to_json_path_ct.values()):
    sample_id = f.split('/CT_')[-1].split('_peak')[0]
    output_folder = label_to_json_ct.get(sample_id).get('output_folder')
    overall_path = '{}/peak_calling/{}_merged_sorted_peaks.fdr_0.1.d_15.scored.tsv'.format(output_folder, sample_id)
    if not os.path.exists(overall_path):
        commands.append(command.format(i+1, f))

lines = header_lines + commands + ["eval ${cmd[$PBS_ARRAYID]}"]
print(commands)
with open('4_RBFOX2_peakcalling_inputs/qsub_rbfox2_ct.sh', 'w') as f:
    for l in lines:
        f.write('{}\n'.format(l))

[]


In [42]:
header_lines = [
    '#!/bin/bash', 
    '#PBS -N peaks_ai_rbfox',
    '#PBS -o peaks_ai_rbfox.sh.out',
    '#PBS -e peaks_ai_rbfox.sh.err',
    '#PBS -V',
    '#PBS -l walltime=10:00:00',
    '#PBS -l nodes=1:ppn=1',
    '#PBS -A yeo-group',
    '#PBS -q home',
    '#PBS -t 1-{}'.format(len(label_to_json_path_ai)),
    '',
    '# Go to the directory from which the script was called',
    'cd /projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/4.3_peakcalling_logs',
    ''
]

command = 'cmd[{}]="module load stamp; snakemake --rerun-incomplete \
--profile /projects/ps-yeolab3/ekofman/sc_STAMP_pipeline/STAMP/profiles/tscc_peakcaller_temp/ \
--configfile /projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/{};"'

commands = []

for i, f in enumerate(label_to_json_path_ai.values()):
    sample_id = f.split('/AI_')[-1].split('_peak')[0]
    output_folder = label_to_json_ai.get(sample_id).get('output_folder')
    overall_path = '{}/peak_calling/{}_merged_sorted_peaks.fdr_0.1.d_15.scored.tsv'.format(output_folder, sample_id)
    if not os.path.exists(overall_path):
        commands.append(command.format(i+1, f))

lines = header_lines + commands + ["eval ${cmd[$PBS_ARRAYID]}"]

print(commands)

with open('4_RBFOX2_peakcalling_inputs/qsub_rbfox2_ai.sh', 'w') as f:
    for l in lines:
        f.write('{}\n'.format(l))

['cmd[13]="module load stamp; snakemake --rerun-incomplete --profile /projects/ps-yeolab3/ekofman/sc_STAMP_pipeline/STAMP/profiles/tscc_peakcaller_temp/ --configfile /projects/ps-yeolab3/ekofman/Hugo/Full_RBFOX2_and_SLBP/4_RBFOX2_peakcalling_inputs/AI_8e_alone_R1_peakcalling.json;"']
